## Installing and importing all libraries for our project

In [1]:
%%capture
%pip install ultralytics;
%pip install -U ipywidgets
%pip install gdown

In [2]:
import os
import sys

from ultralytics import YOLO
import xml.etree.ElementTree as ET
from os import listdir, getcwd
from os.path import join
import shutil
import random
import matplotlib.pyplot as plt
import cv2
import numpy as np

In [3]:
class SuppressOutput:
    def __enter__(self):
        self._stdout = sys.stdout
        self._stderr = sys.stderr
        sys.stdout = open(os.devnull, 'w')
        sys.stderr = open(os.devnull, 'w')
        
    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stderr.close()
        sys.stdout = self._stdout
        sys.stderr = self._stderr

## Downloading and preparing data for training ML-models

### 1. Dataset for human detection

In [4]:
# Downloading dataset for human detection training
!gdown 'https://drive.google.com/u/0/uc?id=1--0QuKMwj31K-CSvD8oq5fceFweiFPuN&export=download'

Downloading...
From (original): https://drive.google.com/u/0/uc?id=1--0QuKMwj31K-CSvD8oq5fceFweiFPuN&export=download
From (redirected): https://drive.google.com/uc?id=1--0QuKMwj31K-CSvD8oq5fceFweiFPuN&export=download&confirm=t&uuid=b07e5c6c-15c0-4d8c-b46c-aa18e1549c97
To: /kaggle/working/human_detection_dataset.zip
100%|███████████████████████████████████████| 2.67G/2.67G [00:20<00:00, 131MB/s]


In [5]:
with SuppressOutput():
    # Unpackaging
    !unzip /kaggle/working/human_detection_dataset.zip
print("Unpacking complete")

Unpacking complete


In [6]:
!rm /kaggle/working/human_detection_dataset.zip
print("The origin zip-file has been deleted")

The origin zip-file has been deleted


### 2. Dataset for mask detection

In [7]:
# Классы объектов (перечислите все классы, которые используются)
classes = ['without_mask', 'mask_weared_incorrect', 'with_mask']

# Функция для конвертации координат
def convert_bbox_to_yolo(size, box):
    if size[0] == 0:
        dw = 1./(size[0]+0.00001)
    else:
        dw = 1./(size[0])
        
    if size[0] == 0:
        dh = 1./(size[1]+0.00001)
    else:
        dh = 1./(size[1])
    x_center = (box[0] + box[1]) / 2.0 - 1
    y_center = (box[2] + box[3]) / 2.0 - 1
    width = box[1] - box[0]
    height = box[3] - box[2]
    return (x_center * dw, y_center * dh, width * dw, height * dh)

# Функция для конвертации XML-аннотации в YOLO-формат
def convert_xml_to_yolo(xml_file, output_dir):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # Получение размера изображения
    size = root.find('size')
    width = int(size.find('width').text)
    height = int(size.find('height').text)

    # Имя изображения
    filename = root.find('filename').text
    output_file = os.path.join(output_dir, os.path.splitext(filename)[0] + '.txt')

    with open(output_file, 'w') as out_file:
        for obj in root.iter('object'):
            cls = obj.find('name').text
            if cls not in classes:
                continue
            cls_id = classes.index(cls)
            xmlbox = obj.find('bndbox')
            bbox = (
                float(xmlbox.find('xmin').text), 
                float(xmlbox.find('xmax').text),
                float(xmlbox.find('ymin').text), 
                float(xmlbox.find('ymax').text)
            )
            bbox_yolo = convert_bbox_to_yolo((width, height), bbox)
            out_file.write(f"{cls_id} {' '.join(map(str, bbox_yolo))}\n")


            
xml_dir = '/kaggle/input/face-mask-detection/annotations'  # The original path to XML-files
image_dir = '/kaggle/input/face-mask-detection/images'  # The original path to XML-files

output_train_dir = '/kaggle/working/face-mask-detection/train'
output_val_dir = '/kaggle/working/face-mask-detection/val'
train_ratio = 0.8  # Соотношение тренировочных данных

os.makedirs(os.path.join(output_train_dir, 'images'), exist_ok=True)
os.makedirs(os.path.join(output_train_dir, 'labels'), exist_ok=True)
os.makedirs(os.path.join(output_val_dir, 'images'), exist_ok=True)
os.makedirs(os.path.join(output_val_dir, 'labels'), exist_ok=True)

# Получение всех XML файлов
xml_files = [f for f in os.listdir(xml_dir) if f.endswith('.xml')]
random.shuffle(xml_files)
# Разделение на тренировочные и валидационные наборы
train_size = int(len(xml_files) * train_ratio)
train_files = xml_files[:train_size]
val_files = xml_files[train_size:]

def move_files(xml_files, dest_dir):
    for xml_file in xml_files:
        image_file = xml_file.replace('.xml', '.png')
        if not os.path.exists(os.path.join(image_dir, image_file)):
            continue
        convert_xml_to_yolo(os.path.join(xml_dir, xml_file), os.path.join(dest_dir, 'labels'))
        shutil.copy(os.path.join(image_dir, image_file), os.path.join(dest_dir, 'images', image_file))

# Перемещаем тренировочные и валидационные файлы
move_files(train_files, output_train_dir)
move_files(val_files, output_val_dir)

print("Convertation and split are completed.")

Convertation and split are completed.


## Trainig models (both are YOLOv8)

In [8]:
human_detection_model = YOLO('yolov8m.pt')
mask_detection_model = YOLO('yolov8m.pt')

100%|██████████| 49.7M/49.7M [00:00<00:00, 234MB/s]


In [9]:
def process_video(model, video_path, output_path):
    cap = cv2.VideoCapture(video_path)
    
    # Получение параметров видео
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    # Создание объекта VideoWriter для записи нового видео с предсказаниями
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Выполнение предсказаний
        results = model(frame)
        
        for result in results:
            # Визуализация результатов для каждого результата
            annotated_frame = result.plot()
        
        # Запись обработанного кадра в новый видеофайл
        out.write(annotated_frame)
    
    # Освобождение ресурсов
    cap.release()
    out.release()

### 1. Training mask detection model

In [10]:
os.environ['WANDB_MODE'] = 'disabled'

result = mask_detection_model.train(
    data='/kaggle/input/mask-data/mask_data.yaml',  # Path to data.yaml
    epochs=35,  # Количество эпох
    imgsz=640,  # Размер изображения
    batch=16,    # Размер батча
)
print("Training of mask detection model complete")

Ultralytics YOLOv8.2.81 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/kaggle/input/mask-data/mask_data.yaml, epochs=1, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, 

100%|██████████| 755k/755k [00:00<00:00, 13.4MB/s]


Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytics

100%|██████████| 6.25M/6.25M [00:00<00:00, 78.2MB/s]


AMP: checks passed ✅


...

Plotting labels to runs/detect/train/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/1      7.16G      2.518      3.952      2.377        129        640:   2%|▏         | 1/43 [00:01<00:57,  1.38s/it]libpng warning: iCCP: Not recognizing known sRGB profile that has been edited
        1/1      7.44G      1.623      1.908      1.392         55        640: 100%|██████████| 43/43 [00:24<00:00,  1.78it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:08<00:00,  1.46s/it]


                   all        171        744      0.417      0.543      0.484      0.285

1 epochs completed in 0.013 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 52.0MB
Optimizer stripped from runs/detect/train/weights/best.pt, 52.0MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.2.81 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 218 layers, 25,841,497 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:07<00:00,  1.26s/it]


                   all        171        744      0.415      0.543      0.484      0.285
          without_mask         57        126      0.517      0.671      0.546      0.284
 mask_weared_incorrect         16         18     0.0544     0.0556     0.0287     0.0177
             with_mask        152        600      0.673      0.903      0.878      0.553
Speed: 0.2ms preprocess, 7.8ms inference, 0.0ms loss, 3.7ms postprocess per image
Results saved to runs/detect/train
Training of mask detection model complete


In [11]:
output_video_path = "output_mask_video.avi"
# Применение модели к видео
process_video(mask_detection_model, "/kaggle/input/test-dataset/crowd_1280_720_30fps.mp4", output_video_path)


0: 384x640 1 without_mask, 3 with_masks, 117.3ms
Speed: 1.8ms preprocess, 117.3ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 without_masks, 4 with_masks, 25.2ms
Speed: 1.7ms preprocess, 25.2ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 without_masks, 5 with_masks, 25.1ms
Speed: 1.6ms preprocess, 25.1ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 without_masks, 3 with_masks, 25.1ms
Speed: 1.6ms preprocess, 25.1ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 without_masks, 2 with_masks, 24.8ms
Speed: 2.0ms preprocess, 24.8ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 without_masks, 3 with_masks, 24.7ms
Speed: 2.1ms preprocess, 24.7ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 without_masks, 4 with_masks, 24.7ms
Speed: 1.6ms preprocess, 24.7ms inference, 1.3ms postprocess per im

### 2. Training human detection model

In [ ]:
os.environ['WANDB_MODE'] = 'disabled'

result = human_detection_model.train(
    data='/kaggle/working/human_detection_dataset/data.yaml',  # Path data.yaml
    epochs=80,  # Количество эпох
    imgsz=640,  # Размер изображения
    batch=16,    # Размер батча
)

In [ ]:
output_video_path = "output_human_video.avi"
# Применение модели к видео
process_video(human_detection_model, "/kaggle/input/test-dataset/crowd_1280_720_30fps.mp4", output_video_path)

## Results

### 1. Result for mask detection model

In [ ]:
result_dir = '/kaggle/working/runs/detect/train'
# Список файлов в папке с результатами
files = os.listdir(result_dir)

if files:
    print("Файлы в папке с результатами:", files)

    for file in files:
        img_path = os.path.join(result_dir, file)
        # Загрузка изображения с использованием OpenCV
        img = cv2.imread(img_path)

        if img is not None:
            # Преобразование изображения в формат RGB для отображения с Matplotlib
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            # Отображение изображения
            plt.figure(figsize=(10, 10))
            plt.imshow(img_rgb)
            plt.axis('off')
            plt.title(file)
            plt.show()
        else:
            print(f"Не удалось загрузить изображение: {file}")
else:
    print("Результаты не были сохранены. Проверьте, правильно ли указан путь.")

In [ ]:
results = mask_detection_model.predict(source='/kaggle/input/mask-data/test.jpeg', save=True)

In [ ]:
result_dir = '/kaggle/working/runs/detect/train3'
# Список файлов в папке с результатами
files = os.listdir(result_dir)

if files:
    # Вывод первых нескольких файлов для проверки
    print("Файлы в папке с результатами:", files)

    for file in files:
        img_path = os.path.join(result_dir, file)
        # Загрузка изображения с использованием OpenCV
        img = cv2.imread(img_path)
        # Преобразование изображения в формат RGB для отображения с Matplotlib
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Отображение изображения
        plt.figure(figsize=(10, 10))
        plt.imshow(img_rgb)
        plt.axis('off')
        plt.title(file)
        plt.show()
else:
    print("Результаты не были сохранены. Проверьте, правильно ли указан путь.")

In [ ]:
results = mask_detection_model.predict(source='/kaggle/input/mask-data/masktypes.jpg', save=True)

In [ ]:
result_dir = '/kaggle/working/runs/detect/train4'
# Список файлов в папке с результатами
files = os.listdir(result_dir)

if files:
    # Вывод первых нескольких файлов для проверки
    print("Файлы в папке с результатами:", files)

    for file in files:
        img_path = os.path.join(result_dir, file)
        # Загрузка изображения с использованием OpenCV
        img = cv2.imread(img_path)
        # Преобразование изображения в формат RGB для отображения с Matplotlib
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Отображение изображения
        plt.figure(figsize=(10, 10))
        plt.imshow(img_rgb)
        plt.axis('off')
        plt.title(file)
        plt.show()
else:
    print("Результаты не были сохранены. Проверьте, правильно ли указан путь.")

### 2. Results for human detection model

In [ ]:
result_dir = '/kaggle/working/runs/detect/train2'
# Список файлов в папке с результатами
files = os.listdir(result_dir)

if files:
    print("Файлы в папке с результатами:", files)

    for file in files:
        img_path = os.path.join(result_dir, file)
        # Загрузка изображения с использованием OpenCV
        img = cv2.imread(img_path)

        if img is not None:
            # Преобразование изображения в формат RGB для отображения с Matplotlib
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            # Отображение изображения
            plt.figure(figsize=(10, 10))
            plt.imshow(img_rgb)
            plt.axis('off')
            plt.title(file)
            plt.show()
        else:
            print(f"Не удалось загрузить изображение: {file}")
else:
    print("Результаты не были сохранены. Проверьте, правильно ли указан путь.")

In [ ]:
results = human_detection_model.predict(source='/kaggle/input/mask-data/test.jpeg', save=True)

In [ ]:
result_dir = '/kaggle/working/runs/detect/train22'
# Список файлов в папке с результатами
files = os.listdir(result_dir)

if files:
    # Вывод первых нескольких файлов для проверки
    print("Файлы в папке с результатами:", files)

    for file in files:
        img_path = os.path.join(result_dir, file)
        # Загрузка изображения с использованием OpenCV
        img = cv2.imread(img_path)
        # Преобразование изображения в формат RGB для отображения с Matplotlib
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Отображение изображения
        plt.figure(figsize=(10, 10))
        plt.imshow(img_rgb)
        plt.axis('off')
        plt.title(file)
        plt.show()
else:
    print("Результаты не были сохранены. Проверьте, правильно ли указан путь.")

In [ ]:
results = human_detection_model.predict(source='/kaggle/input/mask-data/masktypes.jpg', save=True)

In [ ]:
result_dir = '/kaggle/working/runs/detect/train23'
# Список файлов в папке с результатами
files = os.listdir(result_dir)

if files:
    # Вывод первых нескольких файлов для проверки
    print("Файлы в папке с результатами:", files)

    for file in files:
        img_path = os.path.join(result_dir, file)
        # Загрузка изображения с использованием OpenCV
        img = cv2.imread(img_path)
        # Преобразование изображения в формат RGB для отображения с Matplotlib
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Отображение изображения
        plt.figure(figsize=(10, 10))
        plt.imshow(img_rgb)
        plt.axis('off')
        plt.title(file)
        plt.show()
else:
    print("Результаты не были сохранены. Проверьте, правильно ли указан путь.")

In [ ]:
results = human_detection_model.predict(source='/kaggle/working/runs/detect/train4/masktypes.jpg', save=True)

In [ ]:
result_dir = '/kaggle/working/runs/detect/train24'
# Список файлов в папке с результатами
files = os.listdir(result_dir)

if files:
    # Вывод первых нескольких файлов для проверки
    print("Файлы в папке с результатами:", files)

    for file in files:
        img_path = os.path.join(result_dir, file)
        # Загрузка изображения с использованием OpenCV
        img = cv2.imread(img_path)
        # Преобразование изображения в формат RGB для отображения с Matplotlib
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Отображение изображения
        plt.figure(figsize=(10, 10))
        plt.imshow(img_rgb)
        plt.axis('off')
        plt.title(file)
        plt.show()
else:
    print("Результаты не были сохранены. Проверьте, правильно ли указан путь.")